In [1]:
# Program to select the CSV file of pathloss of each indoor wireless environment one-by-one and train the DNN. 

In [2]:
# Importing library

from numpy.random import seed
import tensorflow
from tensorflow import keras

import os
import math
import csv
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, r2_score
from sklearn import model_selection

from sklearn.preprocessing import MinMaxScaler 
from sklearn.decomposition import PCA
#To show the output within the jupyter notebook itself!
%matplotlib inline

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Activation, LeakyReLU, PReLU, ELU, ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import History
from tensorflow.keras import losses

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
# Path of the csv file of power of each indoor environment. For example, indoor_env_26.csv

data_path = r'testing_csv'

In [4]:
# Checking available CSV files 

for i in sorted(os.listdir(data_path),key=len): 
    print(i)

indoor_env_26.csv


In [5]:
# Main program

final_result = {}

test_sizes = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60]  # Different test sizes to save average results 

''' To save average results '''

for i in test_sizes:
    final_result['file_name'] = []
    final_result['mse_train_{}'.format(str(i))] = []
    final_result['mse_test_{}'.format(str(i))] = []
    final_result['mae_train_{}'.format(str(i))] = []
    final_result['mae_test_{}'.format(str(i))] = []
    final_result['No_of_epochs_{}'.format(str(i))] = []

''' To save results for a particular file '''

def main():
    d = {}
    for i in test_sizes:
        d['file_name'] = []
        d['seed'] = []
        d['mse_train_{}'.format(str(i))] = []
        d['mse_test_{}'.format(str(i))] = []
        d['mae_train_{}'.format(str(i))] = []
        d['mae_test_{}'.format(str(i))] = []
        d['No_of_epochs_{}'.format(str(i))] = []
    return d

files = sorted(os.listdir(data_path),key=len)

for idx, file in enumerate(files):
    print(file)
    
    d = main()
    
    seeds = [0, 42] # 2 different seeds
    
    if file.endswith('.csv'):
        
        for s in seeds: 
            
            test_sizes = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60]  # Different test sizes for data split 
            
            for i in test_sizes:
                
                seed(s)
                tensorflow.random.set_seed(s)

                df = pd.read_csv(data_path + '\\' + file, error_bad_lines=False)

                print("Shape of data in CSV file {}: {}".format(idx + 1, df.shape))
                
                ''' Added parts start here for y_pred_all_inv '''
                
                # independent and dependent variables before removing rows having pathloss = 250
                X_actual = df[['X(m)','Y(m)']]      # location of Rxs
                y_actual = df[['Path Loss (dB)']]   # Pathloss
                
                print("Shape of actual input X is {} in data {}".format(X_actual.shape, idx + 1))
                print("Shape of actual output y is {} in data {}".format(y_actual.shape, idx + 1))
                
                ''' Added parts end here for y_pred_all_inv '''

                # removing rows having pathloss = 250. It will drop entire rows.
                df = df[(df['Path Loss (dB)'] != 250)]

                # independent and dependent variables
                X = df[['X(m)','Y(m)']]      # location
                y = df[['Path Loss (dB)']]   # Pathloss        

                print("Shape of input X after replacement is {} in data {}".format(X.shape, idx + 1))
                print("Shape of output y after replacement is {} in data {}".format(y.shape, idx + 1))
                print("Minimum value of {}".format(y.min()))
                print("Maximum value of {}".format(y.max()))

                # converting X and y into numpy array 
                X = X.values
                y = y.values

                # train test split 
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=0)  # data split 

                print("Shape of X_train and X_test is {}, {} in data {}".format(X_train.shape, X_test.shape, idx + 1))
                print("Shape of y_train and y_test is {}, {} in data {}".format(y_train.shape, y_test.shape, idx + 1))
                
                # Scaling input and output 
                scaler1 = MinMaxScaler()
                X_train = scaler1.fit_transform(X_train)
                X_test = scaler1.transform(X_test)
                
                scaler2 = MinMaxScaler()
                y_train = scaler2.fit_transform(y_train)
                y_test = scaler2.transform(y_test)
            
                ''' Added parts start here for y_pred_all_inv '''
                
                X_actual_arr = X_actual.values  # converting dataframe into array
                X_actual_norm = scaler1.fit_transform(X_actual_arr)
                
                ''' Added parts end here for y_pred_all_inv '''

                # Transferring the baseline model to new wireless environments

                # loading saved baseline model from location
                model = load_model(r"no_room_baseline.h5")

                # compiling model 
                model.compile(optimizer=Adam(learning_rate = 0.001), loss='mean_squared_error', metrics = ['mean_absolute_error'])

                # Early Stopping 
                early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=2)

                # Fitting model 
                history = model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping], 
                                batch_size=16, epochs=150)

                # Prediction (Test)
                y_pred = model.predict(X_test)
                
                ''' Test prediction of pathloss starts here for all X_actual '''
                
                y_pred_all = model.predict(X_actual_norm)  # Y before removing pathloss = 250
    
                ''' Test prediction of pathloss ends here for all X_actual '''

                # Test MSE
                mse_test = metrics.mean_squared_error(y_test, y_pred)

                # Test MAE 
                mae_test = metrics.mean_absolute_error(y_test, y_pred)

                # Train MSE
                y_pred_train = model.predict(X_train)
                mse_train = metrics.mean_squared_error(y_train, y_pred_train)

                # Train MAE
                mae_train = metrics.mean_absolute_error(y_train, y_pred_train)

                # Number of epochs
                loss = history.history['loss']
                no_epoch = len(loss)

                # Saving results
                if file not in d['file_name']: d['file_name'].append(file)
                if s not in d['seed']: d['seed'].append(s)
                d['mse_train_{}'.format(str(i))].append(mse_train)
                d['mse_test_{}'.format(str(i))].append(mse_test)
                d['mae_train_{}'.format(str(i))].append(mae_train)
                d['mae_test_{}'.format(str(i))].append(mae_test)
                d['No_of_epochs_{}'.format(str(i))].append(no_epoch)

                print()
                print("*" * 125)
                print()
#                 print(d)
        
        ''' Taking average of results '''      
        for key in final_result.keys():
            if key != 'file_name':
                s = sum(d[key]) / len(d[key])
                final_result[key].append(s)
            elif key == 'file_name': 
                final_result['file_name'].append(d['file_name'][0])

indoor_env_26.csv
Shape of data in CSV file 1: (6678, 7)
Shape of actual input X is (6678, 2) in data 1
Shape of actual output y is (6678, 1) in data 1
Shape of input X after replacement is (6252, 2) in data 1
Shape of output y after replacement is (6252, 1) in data 1
Minimum value of Path Loss (dB)    8.56037
dtype: float64
Maximum value of Path Loss (dB)    91.7336
dtype: float64
Shape of X_train and X_test is (312, 2), (5940, 2) in data 1
Shape of y_train and y_test is (312, 1), (5940, 1) in data 1
Epoch 1/150
20/20 [==============================] - 3s 65ms/step - loss: 0.0081 - mean_absolute_error: 0.0685 - val_loss: 0.0079 - val_mean_absolute_error: 0.0691
Epoch 2/150
20/20 [==============================] - 1s 28ms/step - loss: 0.0080 - mean_absolute_error: 0.0672 - val_loss: 0.0079 - val_mean_absolute_error: 0.0679
Epoch 3/150
20/20 [==============================] - 1s 29ms/step - loss: 0.0078 - mean_absolute_error: 0.0650 - val_loss: 0.0085 - val_mean_absolute_error: 0.0739
E

Epoch 1/150
79/79 [==============================] - 2s 13ms/step - loss: 0.0055 - mean_absolute_error: 0.0558 - val_loss: 0.0053 - val_mean_absolute_error: 0.0540
Epoch 2/150
79/79 [==============================] - 1s 8ms/step - loss: 0.0053 - mean_absolute_error: 0.0550 - val_loss: 0.0051 - val_mean_absolute_error: 0.0533
Epoch 3/150
79/79 [==============================] - 1s 7ms/step - loss: 0.0054 - mean_absolute_error: 0.0557 - val_loss: 0.0054 - val_mean_absolute_error: 0.0530
Epoch 4/150
79/79 [==============================] - 1s 7ms/step - loss: 0.0055 - mean_absolute_error: 0.0557 - val_loss: 0.0052 - val_mean_absolute_error: 0.0548
Epoch 5/150
79/79 [==============================] - 1s 7ms/step - loss: 0.0054 - mean_absolute_error: 0.0563 - val_loss: 0.0054 - val_mean_absolute_error: 0.0555
Epoch 6/150
79/79 [==============================] - 1s 6ms/step - loss: 0.0052 - mean_absolute_error: 0.0544 - val_loss: 0.0051 - val_mean_absolute_error: 0.0525
Epoch 7/150
79/79 [==

Epoch 1/150
118/118 [==============================] - 3s 11ms/step - loss: 0.0046 - mean_absolute_error: 0.0508 - val_loss: 0.0043 - val_mean_absolute_error: 0.0476
Epoch 2/150
118/118 [==============================] - 1s 5ms/step - loss: 0.0045 - mean_absolute_error: 0.0506 - val_loss: 0.0046 - val_mean_absolute_error: 0.0530
Epoch 3/150
118/118 [==============================] - 1s 5ms/step - loss: 0.0045 - mean_absolute_error: 0.0508 - val_loss: 0.0057 - val_mean_absolute_error: 0.0545
Epoch 4/150
118/118 [==============================] - 1s 5ms/step - loss: 0.0045 - mean_absolute_error: 0.0502 - val_loss: 0.0042 - val_mean_absolute_error: 0.0485
Epoch 5/150
118/118 [==============================] - 1s 6ms/step - loss: 0.0043 - mean_absolute_error: 0.0493 - val_loss: 0.0042 - val_mean_absolute_error: 0.0502
Epoch 6/150
118/118 [==============================] - 1s 7ms/step - loss: 0.0045 - mean_absolute_error: 0.0504 - val_loss: 0.0042 - val_mean_absolute_error: 0.0484
Epoch 7/1

Epoch 1/150
157/157 [==============================] - 3s 6ms/step - loss: 0.0043 - mean_absolute_error: 0.0489 - val_loss: 0.0044 - val_mean_absolute_error: 0.0502
Epoch 2/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0043 - mean_absolute_error: 0.0491 - val_loss: 0.0044 - val_mean_absolute_error: 0.0480
Epoch 3/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0043 - mean_absolute_error: 0.0489 - val_loss: 0.0043 - val_mean_absolute_error: 0.0481
Epoch 4/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0041 - mean_absolute_error: 0.0480 - val_loss: 0.0047 - val_mean_absolute_error: 0.0539
Epoch 5/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0042 - mean_absolute_error: 0.0492 - val_loss: 0.0044 - val_mean_absolute_error: 0.0500
Epoch 6/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0042 - mean_absolute_error: 0.0484 - val_loss: 0.0047 - val_mean_absolute_error: 0.0550
Epoch 7/15

Epoch 1/150
59/59 [==============================] - 2s 18ms/step - loss: 0.0080 - mean_absolute_error: 0.0680 - val_loss: 0.0073 - val_mean_absolute_error: 0.0622
Epoch 2/150
59/59 [==============================] - 1s 10ms/step - loss: 0.0074 - mean_absolute_error: 0.0637 - val_loss: 0.0069 - val_mean_absolute_error: 0.0627
Epoch 3/150
59/59 [==============================] - 1s 10ms/step - loss: 0.0075 - mean_absolute_error: 0.0641 - val_loss: 0.0069 - val_mean_absolute_error: 0.0634
Epoch 4/150
59/59 [==============================] - 1s 9ms/step - loss: 0.0075 - mean_absolute_error: 0.0645 - val_loss: 0.0087 - val_mean_absolute_error: 0.0768
Epoch 5/150
59/59 [==============================] - 1s 10ms/step - loss: 0.0077 - mean_absolute_error: 0.0671 - val_loss: 0.0070 - val_mean_absolute_error: 0.0619
Epoch 6/150
59/59 [==============================] - 1s 9ms/step - loss: 0.0075 - mean_absolute_error: 0.0645 - val_loss: 0.0072 - val_mean_absolute_error: 0.0659
Epoch 7/150
59/59 

98/98 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0486 - val_loss: 0.0040 - val_mean_absolute_error: 0.0471
Epoch 23/150
98/98 [==============================] - 1s 5ms/step - loss: 0.0043 - mean_absolute_error: 0.0494 - val_loss: 0.0040 - val_mean_absolute_error: 0.0479
Epoch 24/150
98/98 [==============================] - 1s 5ms/step - loss: 0.0044 - mean_absolute_error: 0.0502 - val_loss: 0.0048 - val_mean_absolute_error: 0.0503
Epoch 25/150
98/98 [==============================] - 0s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0490 - val_loss: 0.0040 - val_mean_absolute_error: 0.0474
Epoch 26/150
98/98 [==============================] - 0s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0489 - val_loss: 0.0042 - val_mean_absolute_error: 0.0468
Epoch 27/150
98/98 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0486 - val_loss: 0.0042 - val_mean_absolute_error: 0.0499
Epoch 28/150
98/98 [=========

Epoch 5/150
137/137 [==============================] - 1s 4ms/step - loss: 0.0042 - mean_absolute_error: 0.0485 - val_loss: 0.0043 - val_mean_absolute_error: 0.0514
Epoch 6/150
137/137 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0494 - val_loss: 0.0042 - val_mean_absolute_error: 0.0480
Epoch 7/150
137/137 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0489 - val_loss: 0.0042 - val_mean_absolute_error: 0.0478
Epoch 8/150
137/137 [==============================] - 1s 4ms/step - loss: 0.0042 - mean_absolute_error: 0.0489 - val_loss: 0.0044 - val_mean_absolute_error: 0.0511
Epoch 9/150
137/137 [==============================] - 1s 4ms/step - loss: 0.0041 - mean_absolute_error: 0.0481 - val_loss: 0.0041 - val_mean_absolute_error: 0.0489
Epoch 10/150
137/137 [==============================] - 1s 4ms/step - loss: 0.0041 - mean_absolute_error: 0.0479 - val_loss: 0.0046 - val_mean_absolute_error: 0.0540
Epoch 11/

157/157 [==============================] - 1s 4ms/step - loss: 0.0040 - mean_absolute_error: 0.0476 - val_loss: 0.0041 - val_mean_absolute_error: 0.0477
Epoch 23/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0473 - val_loss: 0.0042 - val_mean_absolute_error: 0.0502
Epoch 24/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0470 - val_loss: 0.0040 - val_mean_absolute_error: 0.0485
Epoch 25/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0040 - mean_absolute_error: 0.0484 - val_loss: 0.0041 - val_mean_absolute_error: 0.0470
Epoch 26/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0472 - val_loss: 0.0042 - val_mean_absolute_error: 0.0477
Epoch 27/150
157/157 [==============================] - 1s 4ms/step - loss: 0.0039 - mean_absolute_error: 0.0474 - val_loss: 0.0040 - val_mean_absolute_error: 0.0468
Epoch 28/150
157/

In [6]:
# final_result

In [7]:
results = pd.DataFrame(final_result)
results

,file_name,mse_train_0.95,mse_test_0.95,mae_train_0.95,mae_test_0.95,No_of_epochs_0.95,mse_train_0.9,mse_test_0.9,mae_train_0.9,mae_test_0.9,...,mse_train_0.65,mse_test_0.65,mae_train_0.65,mae_test_0.65,No_of_epochs_0.65,mse_train_0.6,mse_test_0.6,mae_train_0.6,mae_test_0.6,No_of_epochs_0.6
0,indoor_env_26.csv,0.007201,0.007937,0.064432,0.067963,11.0,0.007269,0.007008,0.062357,0.062131,...,0.004394,0.004581,0.052518,0.053567,27.0,0.004049,0.004292,0.047961,0.049058,20.5


In [8]:
# saving results in csv file

# results.to_csv(r'testing.csv') 

### Actual pathloss from predicted pathloss after TL 

In [9]:
X_actual_norm

array([[0.        , 0.        ],
       [0.01612903, 0.        ],
       [0.03225806, 0.        ],
       ...,
       [0.96774194, 1.        ],
       [0.98387097, 1.        ],
       [1.        , 1.        ]])

In [10]:
X_actual_norm.shape

(6678, 2)

In [11]:
X_actual_norm.min(), X_actual_norm.max()

(0.0, 1.0000000000000002)

In [12]:
# Predicted pathloss after TL

y_pred_all

array([[0.5401899 ],
       [0.5429117 ],
       [0.5456335 ],
       ...,
       [0.618683  ],
       [0.61871904],
       [0.6187551 ]], dtype=float32)

In [13]:
y_pred_all.shape

(6678, 1)

In [14]:
y_pred_all.min(), y_pred_all.max()

(0.16335438, 0.76461303)

In [16]:
# Actual pathloss from predicted pathloss after TL using min-max scalar

y_pred_all_inv = scaler2.inverse_transform(y_pred_all)
# y_pred_all_inv

In [17]:
y_pred_all_inv.shape

(6678, 1)

In [18]:
y_pred_all_inv.min(), y_pred_all_inv.max()

(22.147081, 72.15571)

In [19]:
# Converting array into dataframe of pathloss

# Actual pathloss from the predicted pathloss after TL 

df_pl = pd.DataFrame(y_pred_all_inv, columns = ['Path Loss (dB)'])
df_pl.head()

,Path Loss (dB)
0,53.489712
1,53.716091
2,53.942471
3,54.265297
4,54.538795


In [20]:
# Saving in csv file. (Actual pathloss obtained from predicted pathloss after TL) 

# df_pl.to_csv(r'actual_pathloss_from_predicted_pathloss_TL_indoor_env_26.csv') 

In [21]:
#done